In [12]:
import serial
import struct
import time

##两个imu分开写
state1 = 0
payload1 = list()

state2 = 0
payload2 = list()

# +SPPDATA=1,028,
header1 = bytes([0x2B, 0x53, 0x50, 0x50, 0x44, 0x41, 0x54, 0x41, 0x3D, 0x31, 0x2C, 0x30, 0x32, 0x38, 0x2C])
# +SPPDATA=2,028,
header2= bytes([0x2B, 0x53, 0x50, 0x50, 0x44, 0x41, 0x54, 0x41, 0x3D, 0x32, 0x2C, 0x30, 0x32, 0x38, 0x2C])

class DataFrame():
    def __init__(self, buffer):
        self.device_addr = buffer[0]
        self.data = buffer[1:25]
        self.invariant = buffer[25:28]
        self.reserved = buffer[28:32]

    def __repr__(self):
        vals = struct.unpack("<6f", self.data)
        vals_str = "{:8.4f},{:8.4f},{:8.4f},{:8.4f},{:8.4f},{:8.4f}".format(vals[0], vals[1], vals[2], vals[3], vals[4], vals[5])
        #invariant_str = "{:02X} {:02X} {:02X}".format(self.invariant[0], self.invariant[1], self.invariant[2])
        #reserved_str = "{:02X} {:02X} {:02X} {:02X}".format(self.reserved[0], self.reserved[1], self.reserved[2], self.reserved[3])
        return vals_str

def getframe1(b):
    global state1
    global payload1

    b = int.from_bytes(b, byteorder='big')
    if state1 == -1 and b == 0x2B:
        state1 = 0
        return False
    elif state1 >= 0 and state1 < 14:
        if header1[state1 + 1] == b:
            state1 = state1 + 1
        else:
            state1 = -1
        return False
    elif state1 >= 14 and state1 < 45:
        payload1.append(b)
        state1 = state1 + 1
        return False
    elif state1 == 45:
        payload1.append(b)
        state1 = -1
        return True

def getframe2(b):
    global state2
    global payload2

    b = int.from_bytes(b, byteorder='big')
    if state2 == -1 and b == 0x2B:
        state2 = 0
        return False
    elif state2 >= 0 and state2 < 14:
        if header2[state2 + 1] == b:
            state2 = state2 + 1
        else:
            state2 = -1
        return False
    elif state2 >= 14 and state2 < 45:
        payload2.append(b)
        state2 = state2 + 1
        return False
    elif state2 == 45:
        payload2.append(b)
        state2 = -1
        return True

In [24]:
def getval(cur):
    whole=[]
    res=cur.split(',')
    for item in res:
        whole.append(eval(item))
    return whole

In [25]:
try:
    port_name = "/dev/tty.usbserial-A50285BI"
    port_bps = 921600
    serial_port = serial.Serial(port_name, port_bps, timeout=1)
    print(serial_port)
    serial_port.write("AT+SPPSEND=1,6,view=1\r\n".encode(encoding="ascii"))
    time.sleep(1)
    serial_port.write("AT+SPPSEND=2,6,view=1\r\n".encode(encoding="ascii"))

    print("command sent")
    f1=open('../data/imu1.txt','w')
    f2=open('../data/imu2.txt','w')
    imu1=[]
    imu2=[]
    
    for i in range(102400):
        data=serial_port.read()
        cur=time.time()
        if getframe1(data):
            frame = DataFrame(bytes(payload1))
            frame = getval(str(frame))
            imu1.append((cur,frame))
            payload1 = []
        if getframe2(data):
            frame = DataFrame(bytes(payload2))
            frame = getval(str(frame))
            imu2.append((cur,frame))
            payload2 = []       
    
    f1.write(str(imu1))
    f2.write(str(imu2))
    
    f1.close()
    f2.close()

except Exception as e:
    print(e)

Serial<id=0x7fdfb1804400, open=True>(port='/dev/tty.usbserial-A50285BI', baudrate=921600, bytesize=8, parity='N', stopbits=1, timeout=1, xonxoff=False, rtscts=False, dsrdtr=False)
command sent


### 数据格式转换
1. 差值
2. 计算角速度

需要注意
- 要把两个imu的时间段统一
- 选择合适的帧率

In [55]:
from scipy.interpolate import interp1d

In [56]:
f1=open('../data/imu1.txt').read()
f2=open('../data/imu2.txt').read()

In [62]:
#看一下两个数据点之间的时间戳
imu1=eval(f1)
tot=0
interval=0
for i in range(1,len(imu1)):
    tot+=1
    interval+=imu1[i][0]-imu1[i-1][0]
    

In [63]:
interval/tot

0.01618837666910401

In [64]:
len(imu1)

1854

In [65]:
len(imu2)

2627

In [66]:
imu1[0]

(1625472245.924957, [71.9774, 18.1563, -15.7351, 0.5672, 0.0817, 0.006])

In [52]:
a=time.time()

In [53]:
b=time.time()

In [54]:
b-a

5.494539022445679